In [37]:
#평가 스케일
def rmsle(y, y_,convertExp=True):
    if convertExp:
        y = np.exp(y), 
        y_ = np.exp(y_)
    log1 = np.nan_to_num(np.array([np.log(v + 1) for v in y]))
    log2 = np.nan_to_num(np.array([np.log(v + 1) for v in y_]))
    calc = (log1 - log2) ** 2
    return np.sqrt(np.mean(calc))

from sklearn import metrics
rmsle_scorer = metrics.make_scorer(rmsle)

# 전처리

In [42]:
import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv("/Users/yoonjeong/Desktop/af_covid_train.csv")
df2 = pd.read_csv("/Users/yoonjeong/Desktop/af_covid_test.csv")

In [44]:
af_train = df.drop(['clustering', 'Unnamed: 0'], axis=1)
af_test = df2.drop(['clustering', 'Unnamed: 0'], axis=1)

모델링

In [46]:
# X : features, y: target variable
X_train=  af_train[af_train.columns.difference(["이용자 수"])]
X_test= af_test[af_test.columns.difference(["이용자 수"])]
y_train= af_train["이용자 수"]
y_test= af_test['이용자 수']

# random forest

In [47]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 10, stop = 30, num = 5)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [10, 15, 20, 25, 30], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [48]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
grid_rf = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1,  scoring=rmsle_scorer)
grid_rf.fit(X_train, y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:918: UserWarning: One or more of the test scores are non-finite: [inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf inf
 inf inf inf inf inf inf inf inf inf inf]
  warnings.warn(
/Users/yoonjeong/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py:925: RuntimeWarning: invalid value encountered in subtract
  array_stds = np.sqrt(np.average((array -


RandomizedSearchCV(cv=10, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [10, 15, 20, 25, 30]},
                   random_state=42, scoring=make_scorer(rmsle), verbose=2)

In [49]:
pred = grid_rf.predict(X_train)
print('RMSLE Value For Random Forest_bf_train: %.4f' % (rmsle(pred,y_train,False)))
print("\n")


y_pred = grid_rf.predict(X_test)
print('RMSLE Value For Random Forest_bf_test : %.4f' % (rmsle(y_pred,y_test,False)))
print("\n")

RMSLE Value For Random Forest_bf_train: 0.1201


RMSLE Value For Random Forest_bf_test : 0.2458




In [50]:
import pandas as pd
rf1 = grid_rf.best_estimator_.fit(X_train, y_train)
feature_imp = pd.DataFrame(rf1.feature_importances_,index=X_train.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_imp

,importance
휴일,0.136694
버스정류장,0.092355
샌드위치,0.077458
은행,0.070963
스타벅스,0.068969
출구,0.058631
인구수,0.055628
확진자수,0.044136
영화관_b,0.041981
노령화지수,0.040666
